# The Eikon API

## How to get Market past data with Python

In the previous episode, we saw how could we get data from a 'live feed', the Eikon server which sends information constantly. We analyzed two methodologies to accomplish our final goal. In this way, we can (for example) update the valuations of the positions we hold and calculate the key metrics of our portfolio every time an updated market price is retrieved. 

This information is very useful if we are building a system that controls our portfolio and generates insights, information day and night. The data retrieval and calculation can run persistently in the background and we can consult it every time we want using a more or less complex interface: a GUI, a Dashboard, or a Web App. We will discuss this aspect in detail in the section regarding presenting data.

We can choose to send updates every X time or at a given hour every day. We can even set thresholds on some key metrics, and send notifications if those limits are breached. But what about more complex metrics/indicators? 

Every asset manager knows that there is one piece of information that is missing and that is vital to perform a more in-depth analysis: data from the past. This is a fundamental piece we need to be able to run practically any kind of financial application, apart from those which are purely based on the present situation of your portfolio. It is sufficient to think about 3 different, yet widely used indicators/metrics:

- moving averages: a mean of past prices.
- volatility: the standard deviation of past returns.
- maximum drawdown: basically the maximum loss over a time span.

All of them use a series of values taken from a past period: it can be one week, one year, or a custom time frame. No matter the time, we must be able to obtain this information for our applications. This is why today's episode is based on how we can fetch past data using Eikon and its Python API.

## How past data is retrieved in Excel?

As we already saw in the introduction, the Eikon interface for Excel is accessible via an Excel COM Add-in during the Eikon terminal's first installation and it is called Datastream. There are two ways of using the library:

1. Browsing the interface that appears on the Ribbon.
2. Typing formulas directly in one cell.

The first method provides an 'easy' way to find the time series you need. It's called _DFO Navigator_ and it's accessed with the button _Find Series_. Clicking the button a GUI will pop up and you can try to find the instruments you need with some filtered searches.

<img src="images/DSButtons.png">

I wrote easy in quotation marks because I always found the interface legacy and hard to navigate. This is only my opinion but you can judge yourself by looking at the example image below. Nowadays we are used to advanced search bars and engines and I found that this particular one would need a refresh. In the example, I tried to search for the S&P500 and that is what I get as the first attempt, without adding filters. Quite disappointing for a company that earns a quite good amount of money from their clients.

<img src="images/DFO_Navigator.png">

Once the instrument you are searching for is found, the _Time Series Request_ Button will help you create the Data request.

<img src="images/DS_TSRequest.png" width="75%">

You can directly type the code in the corresponding field or search for it. You can easily customize your request with a lot of options that are accessed directly on the panel, and this is a very nice feature. 

The result is dumped on the excel spreadsheet in the first column. If you submit more requests at the same time more columns will be filled with data.

<img src="images/DS_TS.png">

If you pay attention to the image above, you will see that the add-in created a custom formula in the first cell: ```DSGRID```, with all the options that were specified during the manual request. Useless to say that if you know already the formula you can type it in one cell and you will still get your data. 

I will finish this Excel review by noticing just one thing: imagine we need to update the time series of all the S&P500 components: they are 500. How would you do it with this manual interface? Well, I will not answer the question because I would like you to come to the same conclusion as me: Read the next section.

## How to retrieve past data in Python

Let's go to the fun part now. As you probably know now, all the processes seen in the previous section can be replicated using the Python Eikon API in a very similar way to what we saw for getting the live data: using a method of the API called ```get_timeseries```.

Here is the description of the method

### ```get_timeseries```

#### Structure

```get_timeseries(rics, fields='*', start_date=None, end_date=None, interval='daily', count=None, calendar=None, corax=None normalize=False, raw_output=False, debug=False)```

#### Parameters

| Name | Type | Description | 
|----- |----- | ----------- |
|rics | string or list of strings | Single RIC or list of RICs to retrieve historical data for |
|start_date | string or datetime.datetime or datetime.timedelta | Starting date and time of the historical range. The string format is ```%Y-%m-%dT%H:%M:%S```, for example, ```2016-01-20T15:04:05```. ```datetime.timedelta``` is negative number of day relative to ```datetime.now()```. The default is ```datetime.now() + timedelta(-100)```.|
|end_date | string or datetime.datetime or datetime.timedelta.  |End date and time of the historical range. Possible string formats: ```%Y-%m-%d```, for example, ```2017-01-20 %Y-%m-%dT%H:%M:%S```, for example, ```2017-01-20T15:04:05 datetime.timedelta``` is a negative number of days relative to ```datetime.now()```. The default is ```datetime.now()```.|
interval | string | Data interval. Possible values are ```tick```, ```minute```, ```hour```, ```daily```, ```weekly```, ```monthly```, ```quarterly```, ```yearly```. The default is ```daily```. |
fields | string or list of strings | Use this parameter to filter the returned fields set. Available fields: ```TIMESTAMP```, ```VALUE```, ```VOLUME```, ```HIGH```, ```LOW```, ```OPEN```, ```CLOSE```, ```COUNT```. By default, all fields are returned. |
count | int, optional | Maximum number of data points retrieved. |
calendar | string, optional | Possible values: ```native```, ```tradingdays```, ```calendardays```. |
corax | string, optional | Possible values: ```adjusted```, ```unadjusted```. |
normalize | boolean | If set to True, the function returns a normalized data frame with the following columns: ```Date```, ```Security```, ```Field```. If the value of this parameter is ```False```, the returned data frame shape depends on the number of RICs and the number of fields in the response. There are three different shapes: One RIC and many fields, many RICs and one field, many RICs, and many fields. The default is ```False```|
raw_output | boolean | Set this parameter to ```True``` to get the data in JSON format. If set to ```False```, the function returns a data frame whose shape is defined by the ```normalize``` parameter. The default is ```False```. |
debug | bool | When set to ```True```, the JSON request and response are printed. The default is ```False```. |

#### Raises

**Exception**
 
If request fails or if server returns an error.

**ValueError**

If a parameter type or value is wrong.

as you can see there are a couple of possibilities to format the input start and end date. You can use a string or the more convenient python ```datetime``` module. If you don't know the module I would suggest you read the documentation [here](https://docs.python.org/3/library/datetime.html), you will need practically always. Let's say we want the information for last week. Either of the following should work:

In [3]:
from datetime import datetime,timedelta

# using timedelta module is easy to obtain 
# a past date in datetime format
start_datetime = datetime.now() + timedelta(-7)
end_datetime = datetime.now()
print(f"start datetime: {start_datetime}")
print(f"end datetime: {end_datetime}")

# the strftime module is able to convert
# a datime to a string of a custom format
start_string = start_datetime.strftime("%Y-%m-%dT%H:%M:%S") 
end_string = end_datetime.strftime("%Y-%m-%dT%H:%M:%S") 
print(f"start string: {start_string}")
print(f"end string: {end_string}")


start datetime: 2022-02-09 16:25:58.615021
end datetime: 2022-02-16 16:25:58.615021
start string: 2022-02-09T16:25:58
end string: 2022-02-16T16:25:58


I used the ```strftime``` method to convert from datetime to convert from ```datetime``` to ```string```. We can now try to retrieve the actual data using those dates. Let's say we want Microsoft and Apple candles (open, close, high, low). The code will be the following:

In [4]:
import eikon

# connect to the eikon api
eikon.set_app_key('your app key here')

# get candle for a couple of instruments
df = eikon.get_timeseries(["MSFT.O","AAPL.O"], 
                        start_date = start_datetime,
                        end_date = end_datetime, 
                        fields = ["OPEN","CLOSE","HIGH","LOW"],
                        interval="daily")

df

Security     MSFT.O                           AAPL.O                         
Field          OPEN    CLOSE    HIGH     LOW    OPEN    CLOSE    HIGH     LOW
Date                                                                         
2022-02-10   304.04   302.38  309.12   300.7  174.14   172.12  175.48  171.55
2022-02-11   303.19   295.04  304.29  294.22  172.33   168.64  173.08  168.04
2022-02-14   293.77    295.0  296.76  291.35  167.37   168.88  169.58  166.56
2022-02-15  300.008   300.47   300.8  297.02  170.97   172.79  172.95  170.25
2022-02-16  298.365  294.475  298.66  293.68  171.85  170.655  172.32  170.05

Two things to notice:
1. it is a multi-index dataframe 
2. the week-end dates are not taken into account

you can access the single dataframes in this way

In [8]:
df["MSFT.O"]

Field,OPEN,CLOSE,HIGH,LOW
Date,,,,
2022-02-10,304.04,302.38,309.12,300.7
2022-02-11,303.19,295.04,304.29,294.22
2022-02-14,293.77,295.0,296.76,291.35
2022-02-15,300.008,300.47,300.8,297.02
2022-02-16,298.365,294.475,298.66,293.68


Some other parameters may be interesting. For the interval, you can go until ```tick```

In [14]:
# get candle for a couple of instruments
df = eikon.get_timeseries(["MSFT.O"], 
                        start_date = start_datetime,
                        end_date = end_datetime, 
                        interval="tick")
df

MSFT.O,VALUE,VOLUME
Date,,
2022-02-16 15:10:35.721,294.43,100
2022-02-16 15:10:35.721,294.41,100
2022-02-16 15:10:35.721,294.4,1
2022-02-16 15:10:35.721,294.42,18
2022-02-16 15:10:35.721,294.42,15
...,...,...
2022-02-16 15:51:12.656,294.9,6
2022-02-16 15:51:12.656,294.9,100
2022-02-16 15:51:12.663,294.895,20


For this only the VALUE and VOLUME fields are present. If you try to query for other fields you will have a DataFrame full of NAs. As you can see, you can easily rump it up to a DataFrame with 50000 rows. The query took 2.5 seconds to run!

You can start playing a bit with the parameters and see what changes, just to have a feeling of how it works to retrieve the data. I promise it will be fun! 🤪

For the moment we saw how to retrieve a stream of data and now you also know how to retrieve past data. Well, I think you are super ready to start building your applications. Once you have the data at hand it's already half of the job. Try to calculate something with the DataFrame you have, like total return or volatility, you will see it's very easy and satisfactory. 

But we're not finished yet. I would like to discuss a couple of more advanced features that you can build in Python, for example how to create a Python Class that will do the job of connecting and retrieving data with yet less effort. What to say: stay with the fellowship! 🧙‍♂️ 